# Hybrid Recommendation System
a) Collaborative Filtering Model

b) Content-based Model

In [71]:
import pandas as pd
from random import randint
import numpy as np
from numpy.linalg import norm 
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from numpy import dot

In [72]:
'''
FORMULA
X(NORMALIZED) = (X - Xminimum)/(Xmaximum - Xminimum)

 min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    
    return [x/max_val for x in data]
'''
#normalizing the input between 0s and 1s
def normalize(data):    
    return (data - np.min(data)/ (np.max(data)- np.min(data)))


In [73]:
''' 
CREATING FUNCTIONS FOR CONTENT BASED RECOMMENDATION MODELS

''' 

#performing one hot encoding
def encoding(df, column):
    new = pd.get_dummies(df[column])
    new.reset_index(drop = True, inplace = True)
    return pd.concat([df, new], axis = 1)

class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        This function will calculate the cosine similarity between two vectors
        '''
        return sum(dot(v1,v2)/(norm(v1)*norm(v2)))
    
    def recommend(self, book_id, n_rec):
        """
        df (dataframe): The dataframe
        song_id (string): Representing the song name
        n_rec (int): amount of rec user wants
        """
        
        # calculate similarity of input book_id vector w.r.t all other vectors
        inputVec = self.df.loc[book_id].values
        self.df['sim']= self.df.apply(lambda x: self.cosine_sim(inputVec, x.values), axis=1)

        # returns top n user specified books
        return self.df.nlargest(columns='sim',n=n_rec)

In [74]:
'''
CREATING FRUNCTION FOR COLLABORATIVE FILTER MODELLING
'''

def svd(mat, df, factors):
    if not 1<= factors < min(mat.shape):
        raise ValueError("Must be 1 <= factors < min(mat.shape)")
    
    #matrix factorization
    u,s,v = svds(mat, k = factors)
    s = np.diag(s)

    #calculating the prediction ratings
    pred = np.dot(np.dot(u,s),v)
    pred = normalize(pred) #normalizing the predictions

    new_df = pd.DataFrame(pred, columns = df.columns, index = list(df.index) ).transpose()

    return new_df


In [75]:
#generating data for various books according to book_id
def generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000):
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d

In [110]:
if __name__ == '__main__':

    df = generate_data(dataset_size = 100000)
    df.to_csv('data.csv', index = False)
    df = df.sort_values(by=['book_id'], ascending = True)
    
    df = df

    # normalizing
    df['num_pages_norm'] = normalize(df['num_pages'].values)
    df['book_rating_norm'] = normalize(df['book_rating'].values)
    df['book_price_norm'] = normalize(df['book_price'].values)
    
    # One hot encoding
    df = encoding(df = df, column = 'publish_year')
    df = encoding(df = df, column = 'book_genre')
    df = encoding(df = df, column = 'text_lang')


    # generate a pivot table
    pivot = df.pivot_table(
        columns = 'book_id',
        index = 'reader_id',
        values = 'book_rating'
    ).fillna(0)

    # convert to a csr matrix
    mat = pivot.values
    mat = csr_matrix(mat)
    
    #applying SVD model
    pred_df = svd(mat, pivot, 10)

    '''
    pred_df[pred_df.select_dtypes(include=['number']).columns] *= 10
    
    pred_df = pred_df.astype(int)
    print(pred_df)  

    '''


    # drop redundant columns
    cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
    df.drop(columns = cols, inplace = True)
    df.set_index('book_id', inplace = True)

    
    # ran on a sample as an example
    t = df.copy()
    cbr = CBRecommend(df = t)


In [119]:
#taking user input
user_input = int(input("Enter the relevant book id: "))

#calling the recommender
data = cbr.recommend(book_id = t.index[0], n_rec = 10000)

#rounding up the values for rating, price, number of pages
data.book_rating_norm = data.book_rating_norm.round(1)
data.book_price_norm = data.book_price_norm.round(2)
data.num_pages_norm = data.num_pages_norm.round()

#calling the user choice
data = data.loc[user_input]

#renaming columns for better vizualization
data.rename(columns = {'num_pages_norm': 'no_of_pages', 'book_rating_norm': 'ratings', 'book_price_norm': 'price'}, inplace =True)

#final output
print_columns = data[['author_id', 'reader_id', 'publisher_id', 'no_of_pages', 'ratings', 'price']]
print_columns

,author_id,reader_id,publisher_id,no_of_pages,ratings,price
book_id,,,,,,
2,317,28872,11,678.0,9.9,176.99
2,124,11524,19,259.0,7.9,89.99
2,345,29841,46,637.0,6.9,175.99
2,192,24509,34,586.0,5.9,168.99
2,113,17789,29,484.0,8.9,149.99
2,179,16180,14,504.0,2.9,165.99
